In [0]:
import warnings
warnings.filterwarnings('ignore')
import pickle
import numpy as np
import pandas as pd
import json
from textblob import TextBlob
import nltk
from scipy import spatial
import torch
import spacy
import string
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
import pdftotext
import os
import time
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import *
from transformers.data.processors.squad import SquadResult, SquadV2Processor, SquadExample
from transformers.data.metrics.squad_metrics import compute_predictions_logits

nltk.download('punkt')
#en_nlp = spacy.load('en_core_web_sm')

[nltk_data] Downloading package punkt to /Users/dharmik/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:

# Loading Squad Dataset

train = pd.read_json("data/train-v1.1.json")
valid = pd.read_json("data/dev-v1.1.json")

In [0]:
# DataFrame of Squad Data

contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(train.shape[0]):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})

In [0]:
# Converting Paragraph into Sentence

s = df["context"].drop_duplicates().reset_index(drop= True).apply(lambda x: [item.raw for item in TextBlob(x).sentences])
sentence = []
for i in range(len(s)):
    for j in range(len(s[i])):
        sentence.append(s[i][j])

In [0]:
import readline
paras = []

class Paragraphs:

    def __init__(self, fileobj, separator='\n'):

        # Ensure that we get a line-reading sequence in the best way possible:
       
        try:
            # Check if the file-like object has an xreadlines method
            self.seq = fileobj.readlines(  )
        except AttributeError:
            # No, so fall back to the xreadlines module's implementation
            self.seq = readlines.readlines(fileobj)

        self.line_num = 0    # current index into self.seq (line number)
        self.para_num = 0    # current index into self (paragraph number)

        # Ensure that separator string includes a line-end character at the end
        
        self.separator = separator


    def __getitem__(self, index):
        if index != self.para_num:
            raise "Only sequential access supported"
        self.para_num += 1
        # Start where we left off and skip 0+ separator lines
        while 1:
        # Propagate IndexError, if any, since we're finished if it occurs
            line = self.seq[self.line_num]
            self.line_num += 1
            if line != self.separator: break
        # Accumulate 1+ nonempty lines into result
        result = [line]
        while 1:
        # Intercept IndexError, since we have one last paragraph to return
            try:
                # Let's check if there's at least one more line in self.seq
                line = self.seq[self.line_num]
            except IndexError:
                # self.seq is finished, so we exit the loop
                break
            # Increment index into self.seq for next time
            self.line_num += 1
            if line == self.separator: break
            result.append(line)
        return ''.join(result)


def show_paragraphs(filename, numpars=5):
    pp = Paragraphs(open(filename))
    for p in pp:
        print ("Par#%d: #%s " % (pp.para_num, repr(p)))
        paras.append(repr(p))

In [0]:
# Loading Test Data From File

### ADD BOOK LOCATION HERE
bookname= 'Marine Diesel Engine'

with open("book1.pdf", "rb") as f:
    pdf = pdftotext.PDF(f)
 
# Save all text to a txt file.
with open(bookname+'.txt', 'w') as f:
    f.write("\n\n".join(pdf))

filename = bookname+'.txt'
show_paragraphs(filename, numpars=5)

Par#1: #'MARINE DIESEL ENGINES\n     THIRD EDITION\n' 
Par#2: #'MARINE DIESEL ENGINES\n Maintenance, Troubleshooting, and Repair\n           THIRD EDITION\n           NIGEL CALDER\n' 
Par#3: #'OTHER BOOKS BY NIGEL CALDER\nBoatowner’s Mechanical and Electrical Manual: How to Maintain, Repair and\n   Improve Your Boat’s Essential Systems, Third Edition\nThe Cruising Guide to the Northwest Caribbean: The Yucatan Coast of Mexico,\n   Belize, Guatemala, Honduras, and the Bay Islands of Honduras\nCuba: A Cruising Guide\nHow to Read a Nautical Chart: A Complete Guide to the Symbols,\n   Abbreviations, and Data Displayed on Nautical Charts\nNigel Calder’s Cruising Handbook: A Compendium for Coastal and Offshore\n   Sailors\nRefrigeration for Pleasureboats: Installation, Maintenance, and Repair Repairs\n   at Sea\nCopyright © 1987, 1992 by International Marine & 2007 by Nigel Calder. All\nrights reserved. Except as permitted under the United States Copyright Act of\n1976, no part of this public

In [0]:
# Converting Test Data to Paragraph

list_para=[]
for p in paras:
    p =p[1:-4]
    p = p.replace('\\n',' ')
    p = p.replace('\\uf0b7',' ')
    p = p.replace('  ',' ')
    p = p.replace('  ',' ')
    p = p.replace('  ',' ')
    p = p.replace('  ',' ')
    p = p.replace('  ',' ')
    p = p.replace(bookname,'')
    
    list_para.append(p[1:-4])
print(list_para)

['ARINE DIESEL ENGINES THIRD ED', 'ARINE DIESEL ENGINES Maintenance, Troubleshooting, and Repair THIRD EDITION NIGEL C', 'THER BOOKS BY NIGEL CALDER Boatowner’s Mechanical and Electrical Manual: How to Maintain, Repair and Improve Your Boat’s Essential Systems, Third Edition The Cruising Guide to the Northwest Caribbean: The Yucatan Coast of Mexico, Belize, Guatemala, Honduras, and the Bay Islands of Honduras Cuba: A Cruising Guide How to Read a Nautical Chart: A Complete Guide to the Symbols, Abbreviations, and Data Displayed on Nautical Charts Nigel Calder’s Cruising Handbook: A Compendium for Coastal and Offshore Sailors Refrigeration for Pleasureboats: Installation, Maintenance, and Repair Repairs at Sea Copyright © 1987, 1992 by International Marine & 2007 by Nigel Calder. All rights reserved. Except as permitted under the United States Copyright Act of 1976, no part of this publication may be reproduced or distributed in any form or by any means, or stored in a database or retrie

In [0]:
# Converting Paragraph to Sentence

data = pd.DataFrame(list_para)
data = data.rename(columns={0:'para'})
data['sentences'] = data['para'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])

list_sent = []
para_no=[]
for i in range(len(data['sentences'])):
    for j in range(len(data['sentences'][i])):
        list_sent.append(data['sentences'][i][j])
        para_no.append(i)

In [0]:
print(data['sentences'][12])

['PREFACE TO THE THIRD EDITION More than ten years have gone by since I wrote the second edition of this book.', 'In this time, there have been significant changes in the diesel engine world.', 'Whereas in the past such changes have occurred largely as a result of economic and competitive pressures, in recent years the motivation for change has come from what is known as “technology forcing” legislation, primarily in the form of ever-tightening emissions standards (see the Technology Forcing Legislation sidebar next page).', 'When such legislation is first introduced, many in the industry argue that the new standards will be impossible to meet, but in fact, as each successive deadline has approached, manufacturers have invariably succeeded in exceeding the new requirements.', 'Some will admit off the record that the legislative pressure has been good for the industry.', 'When first proposed, most of the tightened standards were not applicable to marine engines.', 'But because of the re

In [0]:
# Loading InferSent Model and Building Vocab

sentence.extend(list_sent)

from models_infersent import InferSent
V = 1
MODEL_PATH = 'InferSent/encoder/infersent%s.pkl' % V
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': V}
infersent = InferSent(params_model)
infersent.load_state_dict(torch.load(MODEL_PATH))
W2V_PATH = 'InferSent/dataset/GloVe/glove.840B.300d.txt'
infersent.set_w2v_path(W2V_PATH)
infersent.build_vocab(sentence, tokenize=True)

Found 89006(/109700) words with w2v vectors
Vocab size : 89006


In [0]:
# Function to find number of Words

def func(ques):
    word=1
    ques=str(ques)
    for i in ques:
        if(i==' '):
            word=word+1
    return word

In [0]:
# Cleaning of the Question entered by the User
### ENTER QUESTION HERE
ques = 'Where were large steam engines used?'
ques=ques.strip()
a=ques.rindex("?")
ques=ques.replace(ques[a],"").strip()
word_q=func(ques)
if(word_q>64):
    flag_q=1
print(ques)
dict_embeddings = {}
dict_embeddings[ques] = infersent.encode([ques], tokenize=True)

Where were large steam engines used


In [0]:
print(dict_embeddings[ques].shape)

(1, 4096)


In [0]:
# Preprocessing the Question  

def preprocess(x): #Using TextBlob
    exclude = set(string.punctuation)
    x =''.join(ch for ch in x if ch not in exclude)
    
    lower = x.lower()
    
    def lemmatize_with_postag(sentence):
        sent = TextBlob(sentence)
        tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
        words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sent.tags]    
        lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
        ans = " ".join(lemmatized_list)
        return ans


    lem_sentence = lemmatize_with_postag(lower)
    
    tokens = word_tokenize(lem_sentence)
    
    stop_words = set(stopwords.words('english'))
    nostop = [w for w in tokens if not w in stop_words]
    
    return nostop

pre_words = preprocess(ques)    

In [0]:
pre_words

['large', 'steam', 'engine', 'use']

In [0]:
# Searching Parameters in the Corpus 

x=0
pre_para = []
para_data =[]
for a in list_para:
    for w in pre_words:
        if w in a:
            pre_para.append(x)
            para_data.append(list_para[x])
            break
    x += 1

In [0]:
para_data

['ï»¿The steam engine was one of the most important inventions of the Industrial Revolution. Steam engines were used in all sorts of applications including factories, mines, locomotives, and steamboats',
 'Steam engines use hot steam from boiling water to drive a piston (or pistons) back and forth. The movement of the piston was then used to power a machine or turn a wheel. To create the steam, most steam engines heated the water by burning coal.',
 "The steam engine helped to power the Industrial Revolution. Before steam power, most factories and mills were powered by water, wind, horse, or man. Water was a good source of power, but factories had to be located near a river. Both water and wind power could be unreliable as sometimes rivers could dry up during a drought or freeze during the winter and wind didn't always blow",
 'Steam power allowed for factories to be located anywhere. It also provided reliable power and could be used to power large machines',
 "One of the first steam e

In [0]:
pre_para

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 16, 17, 21, 22, 23, 24]

In [0]:
# Converting the Selected Paragraphs into Sentences

pre_sent = []
pre_para_id = []
for i in range(0,len(para_data)):
    x = TextBlob(para_data[i]).sentences
    for a in x:
        pre_sent.append(str(a))
        pre_para_id.append(pre_para[i])
    

In [0]:
pre_sent

['ï»¿The steam engine was one of the most important inventions of the Industrial Revolution.',
 'Steam engines were used in all sorts of applications including factories, mines, locomotives, and steamboats',
 'Steam engines use hot steam from boiling water to drive a piston (or pistons) back and forth.',
 'The movement of the piston was then used to power a machine or turn a wheel.',
 'To create the steam, most steam engines heated the water by burning coal.',
 'The steam engine helped to power the Industrial Revolution.',
 'Before steam power, most factories and mills were powered by water, wind, horse, or man.',
 'Water was a good source of power, but factories had to be located near a river.',
 "Both water and wind power could be unreliable as sometimes rivers could dry up during a drought or freeze during the winter and wind didn't always blow",
 'Steam power allowed for factories to be located anywhere.',
 'It also provided reliable power and could be used to power large machines'

In [0]:
pre_para_id

[0,
 0,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 3,
 3,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 10,
 10,
 10,
 11,
 11,
 13,
 13,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 17,
 17,
 17,
 17,
 17,
 17,
 17,
 21,
 21,
 21,
 21,
 22,
 22,
 23,
 23,
 23,
 24,
 24]

In [0]:
# Converting Sentences to Embeddings

for i in range(len(pre_sent)):
    dict_embeddings[pre_sent[i]] = infersent.encode([pre_sent[i]], tokenize=True)
    print(dict_embeddings[pre_sent[i]])

[[ 0.04380202  0.16262217  0.0710826  ...  0.01979169 -0.03814263
   0.07085073]]
[[ 0.08162108  0.12036797  0.05120068 ...  0.11094005 -0.03814263
   0.03642646]]
[[ 0.04150315  0.12036797  0.13460205 ... -0.0338376   0.07153036
   0.02382679]]
[[ 0.00456284  0.11647291  0.08575004 ... -0.03926745 -0.01222112
   0.01436585]]
[[0.06844595 0.11854672 0.07675788 ... 0.06818925 0.02050814 0.05533498]]
[[ 0.02272708  0.1420885   0.05174219 ... -0.03926747 -0.00081242
  -0.00485754]]
[[0.06412293 0.10373987 0.05798866 ... 0.07479809 0.01041548 0.08345925]]
[[ 0.10550181  0.11328059  0.06789432 ... -0.02247149 -0.02051079
  -0.00270376]]
[[-0.01052266  0.08589423  0.1320693  ...  0.03789843  0.01326812
   0.04653865]]
[[ 0.12369134  0.08616728  0.06447218 ... -0.03176411 -0.03814263
  -0.02892261]]
[[ 0.04815323  0.06155445  0.03523442 ...  0.018139   -0.01792006
  -0.02674313]]
[[ 0.0304859   0.12043131  0.01910035 ... -0.0015618  -0.03814263
   0.0966199 ]]
[[ 0.01451322  0.03671872  0.072

In [0]:
# Function to compute Cosine Similarity

def cosine_sim(pre_sent,ques):
    li = []
    for item in range(0,len(pre_sent)):
        li.append(spatial.distance.cosine(dict_embeddings[pre_sent[item]],dict_embeddings[ques]))
    return li

In [0]:
cosine_sim(pre_sent,ques)

[0.3602234721183777,
 0.2883431315422058,
 0.3166273236274719,
 0.40255260467529297,
 0.27828800678253174,
 0.3270326852798462,
 0.34221845865249634,
 0.45782798528671265,
 0.4525049924850464,
 0.42328429222106934,
 0.3476157784461975,
 0.3935398459434509,
 0.4991944432258606,
 0.3978303074836731,
 0.3287000060081482,
 0.4951232671737671,
 0.2905768156051636,
 0.31724023818969727,
 0.3214028477668762,
 0.2722044587135315,
 0.45394235849380493,
 0.2148248553276062,
 0.2200394868850708,
 0.32262009382247925,
 0.28797852993011475,
 0.33647531270980835,
 0.33958911895751953,
 0.43631088733673096,
 0.4070896506309509,
 0.38643085956573486,
 0.5278308987617493,
 0.4655880331993103,
 0.4151444435119629,
 0.4369102120399475,
 0.520306259393692,
 0.3834916353225708,
 0.47499358654022217,
 0.41921311616897583,
 0.4447000026702881,
 0.4402705430984497,
 0.3658010959625244,
 0.5264796614646912,
 0.5342124402523041,
 0.5910631418228149,
 0.5568555295467377,
 0.5629464983940125,
 0.5845688581466675,

In [0]:
# Function to return Sentence with minimum Cosine Distance

def para_idx(pre_sent,ques):
    li = cosine_sim(pre_sent,ques)
    min = np.argmin(li)
    return pre_para_id[min]

a = para_idx(pre_sent,ques)

In [0]:
def para_idx(pre_sent,ques,n):
    distances = cosine_sim(pre_sent,ques)
    li =[]
    count = 0
    while count < 10:
        min = np.argmin(distances)
        distances[min]+=1
        if pre_para_id[min] not in li :
            li.append(pre_para_id[min])
            count+=1
    return li

n=3
a = para_idx(pre_sent,ques,n)

In [0]:
print(a)

[6, 1, 7, 0, 5, 17, 2, 4, 8, 3]


In [0]:
# Config
n_best_size = 1

max_answer_length = 30
do_lower_case = True
null_score_diff_threshold = 0.0

In [0]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [0]:
# Loading the  AlBert Model

import pickle
f=open('store3.pckl','rb')
model=pickle.load(f)
f.close()

In [0]:
b = 'albert-xxlarge-spiece.model'

In [0]:
f=open('store6.pckl','rb')
tokenizer_class=pickle.load(f)
f.close()

In [0]:
# Loading the Tokenizer

tokenizer = tokenizer_class.from_pretrained(b,do_lower_case=True)

Calling AlbertTokenizer.from_pretrained() with the path to a single file or url is deprecated


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
processor = SquadV2Processor()

In [0]:
# Function to Run Predictions using AlBert

def run_prediction(question_texts, context_text):
    """Setup function to compute predictions"""
    examples = []

    for i, question_text in enumerate(question_texts):
        example = SquadExample(
            qas_id=str(i),
            question_text=question_text,
            context_text=context_text,
            answer_text=None,
            start_position_character=None,
            title="Predict",
            is_impossible=False,
            answers=None,
        )

        examples.append(example)

    features, dataset = squad_convert_examples_to_features(
        examples=examples,
        tokenizer=tokenizer,
        max_seq_length=512,
        doc_stride=128,
        max_query_length=64,
        is_training=False,
        return_dataset="pt",
        threads=1,
    )

    eval_sampler = SequentialSampler(dataset)
    eval_dataloader = DataLoader(dataset, sampler=eval_sampler, batch_size=10)

    all_results = []

    for batch in eval_dataloader:
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }

            example_indices = batch[3]

            outputs = model(**inputs)

            for i, example_index in enumerate(example_indices):
                eval_feature = features[example_index.item()]
                unique_id = int(eval_feature.unique_id)

                output = [to_list(output[i]) for output in outputs]

                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
                all_results.append(result)

    output_prediction_file = "predictions.json"
    output_nbest_file = "nbest_predictions.json"
    output_null_log_odds_file = "null_predictions.json"

    predictions = compute_predictions_logits(
        examples,
        features,
        all_results,
        n_best_size,
        max_answer_length,
        do_lower_case,
        output_prediction_file,
        output_nbest_file,
        output_null_log_odds_file,
        False,  # verbose_logging
        True,  # version_2_with_negative
        null_score_diff_threshold,
        tokenizer,
    )

    return predictions

In [0]:
context = []
for i in a:
    context.append(list_para[i])
    print(list_para[i])
    
questions = [ques]

Throughout the 1800s, steam engines were improved. They became smaller and more efficient. Large steam engines were used in factories and mills to power machines of all types. Smaller steam engines were used in transportation including trains and steamboats
Steam engines use hot steam from boiling water to drive a piston (or pistons) back and forth. The movement of the piston was then used to power a machine or turn a wheel. To create the steam, most steam engines heated the water by burning coal.
The steam engine as we think of it from the Industrial Revolution was largely replaced by electricity and the internal combustion engine (gas and diesel). Some old steam engines are still used in certain areas of the world and in antique locomotives
ï»¿The steam engine was one of the most important inventions of the Industrial Revolution. Steam engines were used in all sorts of applications including factories, mines, locomotives, and steamboats
Steam power really took off with improvements m

In [0]:
len(context)

10

In [0]:
list_para[6]

['ï»¿The steam engine was one of the most important inventions of the Industrial Revolution. Steam engines were used in all sorts of applications including factories, mines, locomotives, and steamboats',
 'Steam engines use hot steam from boiling water to drive a piston (or pistons) back and forth. The movement of the piston was then used to power a machine or turn a wheel. To create the steam, most steam engines heated the water by burning coal.',
 "The steam engine helped to power the Industrial Revolution. Before steam power, most factories and mills were powered by water, wind, horse, or man. Water was a good source of power, but factories had to be located near a river. Both water and wind power could be unreliable as sometimes rivers could dry up during a drought or freeze during the winter and wind didn't always blow",
 'Steam power allowed for factories to be located anywhere. It also provided reliable power and could be used to power large machines',
 "One of the first steam e

In [0]:
# Finding the Precise answer

ans=[]
for i in context:
    flag=0
    word_c=func(i)
    if(word_c>511):
        flag=1
    else:
    # Run method
        predictions = run_prediction(questions, i)
        for key in predictions.keys():
            if(predictions[key]!=""): ans.append(predictions[key])

add example index and unique id: 100%|██████████| 1/1 [00:00<?, ?it/s]


In [0]:
ans

['Large steam engines were used in factories and mills to power machines of all types',
 'Some old steam engines are still used in certain areas of the world and in antique locomotives',
 'factories, mines, locomotives, and steamboats',
 'pump water out of mines.The Porter-Allen high-speed steam engine was popular during the late 1800s and early 1900s',
 'around the world in various applications',
 'factories to be located anywhere. It also provided reliable power and could be used to power large machines']